## Step 1: Data Preparation

In [3]:
import os

print(f"Previous exec path: {os.getcwd()}")
# move two level up
os.chdir("../../")
print(f"Current exec path: {os.getcwd()}")

from source.utils.dot2polygon import dot2polygon
import glob
import yaml

Previous exec path: /workspace/source/notebooks
Current exec path: /workspace


In [2]:
# make a folder for converted annotations
output_root = r"./data/monkey-data/annotations_polygon"
if not (os.path.isdir(output_root)):
    os.mkdir(output_root)

Convert the dot annotation to polygon bounding boxes

In [3]:
# TODO: check those values because in the eval they changed the numbers!
lymphocyte_half_box_size = (
    4.5  # the size of half of the bbox around the lymphocyte dot in um
)
# NOTE: reduced this to 5.0 as the eval script (it was 11.0)
monocytes_half_box_size = (
    5.0  # the size of half of the bbox around the monocytes dot in um
)
min_spacing = 0.25  # NOTE: in the eval code they use 0.24199951445730394
# spacing is the zoom level of the image, in micro-meters per pixel (was rounded to 0.25)

In [5]:
# convert dot annotations to polygon, single image

# xml_path = r'./data/monkey-data/annotations/xml/A_P000001.xml'
# output_path = r'./data/monkey-data/annotations_polygon/A_P000001_polygon.xml'

# convert the dot annotations (in mm) to polygon bounding boxes
# dot2polygon(xml_path, lymphocyte_half_box_size, monocytes_half_box_size, min_spacing, output_path)

In [6]:
# convert dot annotations to polygon, in a folder

annotation_dir = r"./data/monkey-data/annotations/xml"
annotation_polygon_dir = r"./data/monkey-data/annotations_polygon"
annotation_list = glob.glob(os.path.join(annotation_dir, "*.xml"))

print(annotation_list)
for xml_path in annotation_list:
    print(xml_path)
    output_path = os.path.join(
        annotation_polygon_dir,
        os.path.splitext(os.path.basename(xml_path))[0]
        + "_polygon"
        + os.path.splitext(os.path.basename(xml_path))[1],
    )
    print(output_path)
    # lymphocyte_half_box_size = 4.5 # the size of half of the bbox around the lymphocyte dot in um
    # monocytes_half_box_size = 11.0 # the size of half of the bbox around the monocytes dot in um
    # min_spacing = 0.25
    dot2polygon(
        xml_path,
        lymphocyte_half_box_size,
        monocytes_half_box_size,
        min_spacing,
        output_path,
    )

['./data/monkey-data/annotations/xml/D_P000018.xml', './data/monkey-data/annotations/xml/B_P000019.xml', './data/monkey-data/annotations/xml/A_P000033.xml', './data/monkey-data/annotations/xml/D_P000012.xml', './data/monkey-data/annotations/xml/D_P000013.xml', './data/monkey-data/annotations/xml/A_P000017.xml', './data/monkey-data/annotations/xml/A_P000030.xml', './data/monkey-data/annotations/xml/C_P000022.xml', './data/monkey-data/annotations/xml/C_P000021.xml', './data/monkey-data/annotations/xml/C_P000036.xml', './data/monkey-data/annotations/xml/B_P000016.xml', './data/monkey-data/annotations/xml/D_P000011.xml', './data/monkey-data/annotations/xml/C_P000023.xml', './data/monkey-data/annotations/xml/A_P000004.xml', './data/monkey-data/annotations/xml/D_P000010.xml', './data/monkey-data/annotations/xml/B_P000014.xml', './data/monkey-data/annotations/xml/A_P000020.xml', './data/monkey-data/annotations/xml/B_P000002.xml', './data/monkey-data/annotations/xml/C_P000033.xml', './data/mon

Creating the yaml config files for training.

In [7]:
def folders_to_yml(wsi_dir: str, wsa_dir: str, output_dir: str, output_name: str):
    """
    Generate a yaml file to be used as WSD dataconfig from a folder of slides and a folder of annotation or mask files.
    Assumes files use the same name for both the slides and masks.
    """

    wsa_list = glob.glob(wsa_dir)

    yaml_dict = {"training": []}
    # yaml_dict = {'training': [], 'validation': []}
    for wsa in wsa_list:
        patient_name = os.path.basename(wsa).split(
            os.path.basename(wsa_dir).split("*")[1]
        )[0]  # monocytes
        #     print(patient_name)
        if os.path.isfile(os.path.join(wsi_dir, patient_name + "_PAS_CPG.tif")):
            wsi = os.path.join(wsi_dir, patient_name + "_PAS_CPG.tif")
            print("match found:    ", patient_name)
            yaml_dict["training"].append(
                {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}}
            )

            # # validation if needed
            # yaml_dict['validation'].append(
            #         {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}})

        else:
            print("no match found:    ", patient_name)

    # make a folder for output
    if not (os.path.isdir(output_dir)):
        os.mkdir(output_dir)

    with open(os.path.join(output_dir, output_name), "w") as file:
        yaml.safe_dump(yaml_dict, file)

In [8]:
wsi_dir = r"./data/monkey-data/images/pas-cpg"  # r'./data_debug/images/pas-cpg'
wsa_dir = r"./data/monkey-data/annotations_polygon/*_polygon.xml"
output_dir = r"./source/configs"
output_name = "training_full.yml"

folders_to_yml(wsi_dir, wsa_dir, output_dir, output_name)

match found:     B_P000004
match found:     D_P000011
match found:     B_P000005
match found:     A_P000036
match found:     A_P000001
match found:     D_P000004
match found:     A_P000011
match found:     C_P000031
match found:     A_P000029
match found:     D_P000010
match found:     C_P000033
match found:     A_P000035
match found:     D_P000015
match found:     C_P000036
match found:     A_P000006
match found:     A_P000018
match found:     B_P000003
match found:     B_P000016
match found:     A_P000034
match found:     C_P000037
match found:     D_P000002
match found:     B_P000007
match found:     B_P000015
match found:     D_P000001
match found:     D_P000009
match found:     A_P000004
match found:     C_P000023
match found:     A_P000003
match found:     A_P000005
match found:     A_P000014
match found:     C_P000035
match found:     C_P000024
match found:     D_P000016
match found:     C_P000038
match found:     B_P000020
match found:     D_P000007
match found:     C_P000039
m

In [15]:
import yaml
import os
import pprint

config_path = "./source/configs/baseline/detectron2_baseline.yml"

if os.path.exists(config_path):
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)
        print(f"Loaded configuration from {config_path}:")
        config_formatted = pprint.pformat(config, indent=4)
        print(config_formatted)
else:
    print(
        f"Configuration file {config_path} does not exist.\nLoading default config file."
    )
    config = {}

Loaded configuration from ./source/configs/baseline/detectron2_baseline.yml:
{   'dataset': {   'annotation_polygon_dir': 'annotations_polygon',
                   'lymphocyte_half_box_size': 4.5,
                   'min_spacing': 0.25,
                   'monocytes_half_box_size': 5.0,
                   'name': 'monkey_dataset',
                   'path': '../data/monkey-data',
                   'yaml_wsi_wsa_path': './configs/training_full.yml'},
    'model': {'name': 'detectron2'},
    'optimizer': None,
    'other': None,
    'project': {   'continue_training': False,
                   'file_log': False,
                   'img_save_dir_path': './imgs',
                   'log_dir': '../logs/scripts',
                   'model_save_dir_path': None,
                   'name': 'monkey_challenge_ziplab',
                   'output_dir': '../results',
                   'seed': 42,
                   'timestamp': 'auto',
                   'wandb_model_watch': True},
    'training':

In [19]:
config.get("dataset", {})

{'name': 'monkey_dataset',
 'path': '../data/monkey-data',
 'annotation_polygon_dir': 'annotations_polygon',
 'yaml_wsi_wsa_path': './configs/training_full.yml',
 'lymphocyte_half_box_size': 4.5,
 'monocytes_half_box_size': 5.0,
 'min_spacing': 0.25}

In [18]:
user_config = config["wholeslidedata"]["user_config"]
user_config["wholeslidedata"]["default"]["yaml_source"] = 

{'wholeslidedata': {'default': {'yaml_source': './configs/training_full.yml',
   'seed': 42,
   'image_backend': 'asap',
   'labels': {'ROI': 0, 'lymphocytes': 1, 'monocytes': 2},
   'batch_shape': {'batch_size': 10,
    'spacing': 0.5,
    'shape': [128, 128, 3],
    'y_shape': [1000, 6]},
   'annotation_parser': {'sample_label_names': ['roi']},
   'point_sampler_name': 'RandomPointSampler',
   'point_sampler': {'buffer': {'spacing': '${batch_shape.spacing}',
     'value': -64}},
   'patch_label_sampler_name': 'DetectionPatchLabelSampler',
   'patch_label_sampler': {'max_number_objects': 1000,
    'detection_labels': ['lymphocytes', 'monocytes']}}}}